# Anthropic Claude Tutorial

This notebook covers working with Anthropic's Claude models using the `llm_playbook` package.

## What You'll Learn

- Setting up the Anthropic client
- Basic messages API usage
- System prompts (Claude's specialty!)
- Multi-turn conversations
- Streaming responses

## Available Models

| Model | Description |
|-------|-------------|
| `claude-sonnet-4-20250514` | Latest Sonnet - balanced (default) |
| `claude-opus-4-20250514` | Most capable, best reasoning |
| `claude-3-5-sonnet-20241022` | Previous Sonnet version |
| `claude-3-haiku-20240307` | Fast and affordable |

## Why Claude?

- Excellent at following complex instructions
- Strong reasoning capabilities
- 200K token context window
- Great for analysis and writing tasks

## Setup

Install the package and configure your API key.

In [ ]:
# Install the package
!pip install -q git+https://github.com/deepakdeo/python-llm-playbook.git

In [ ]:
# Setup API Key from Colab Secrets
import os
from google.colab import userdata

# Add your ANTHROPIC_API_KEY in the Secrets pane (🔑 icon in left sidebar)
os.environ['ANTHROPIC_API_KEY'] = userdata.get('ANTHROPIC_API_KEY')
print("API key configured!")

## 1. Basic Messages API

The simplest way to use Claude - send a message and get a response.

In [ ]:
from llm_playbook import AnthropicClient

# Initialize the client (uses claude-sonnet-4 by default)
client = AnthropicClient()

# Simple chat
response = client.chat("What is machine learning in one sentence?")
print(response)

In [ ]:
# Use Claude 3 Haiku for faster, cheaper responses
haiku_client = AnthropicClient(model="claude-3-haiku-20240307")

response = haiku_client.chat("What is 2 + 2?")
print(f"Haiku says: {response}")

## 2. System Prompts

Claude excels at following system prompts. This is where you define its behavior, personality, and constraints.

In [ ]:
# Without system prompt
response = client.chat("Review this code: print('hello world')")
print("Without system prompt:")
print(response)
print()

In [ ]:
# With detailed system prompt
response = client.chat(
    message="Review this code: print('hello world')",
    system_prompt="""You are a senior Python code reviewer. For each code snippet:
1. Identify any issues or improvements
2. Rate it on a scale of 1-10
3. Provide a brief suggestion

Be constructive but honest."""
)
print("With code reviewer persona:")
print(response)

In [ ]:
# Claude as a specific character
response = client.chat(
    message="Tell me about space exploration",
    system_prompt="You are Carl Sagan. Speak with wonder about the cosmos, use poetic language, and reference the 'pale blue dot' perspective."
)
print(response)

## 3. Multi-turn Conversations

Maintain context across multiple exchanges. Claude has excellent memory within conversations.

In [ ]:
from llm_playbook import ChatMessage

# Initialize conversation
history = []
system = "You are a helpful math tutor. Explain concepts step by step."

# Turn 1
q1 = "What is a derivative in calculus?"
a1 = client.chat(q1, system_prompt=system, history=history)

print(f"Student: {q1}")
print(f"Tutor: {a1}\n")

history.append(ChatMessage(role="user", content=q1))
history.append(ChatMessage(role="assistant", content=a1))

In [ ]:
# Turn 2 - follows up naturally
q2 = "Can you give me a simple example?"
a2 = client.chat(q2, system_prompt=system, history=history)

print(f"Student: {q2}")
print(f"Tutor: {a2}\n")

history.append(ChatMessage(role="user", content=q2))
history.append(ChatMessage(role="assistant", content=a2))

In [ ]:
# Turn 3 - Claude remembers the context
q3 = "What about the derivative of x squared?"
a3 = client.chat(q3, system_prompt=system, history=history)

print(f"Student: {q3}")
print(f"Tutor: {a3}")

## 4. Streaming Responses

Stream tokens as they're generated. Essential for long responses and chat interfaces.

In [ ]:
print("Streaming: ", end="")

for token in client.stream("Write a haiku about artificial intelligence."):
    print(token, end="", flush=True)

print()

In [ ]:
# Streaming a longer response
print("Streaming story: ", end="")

for token in client.stream(
    message="Write a very short story (3 sentences) about a robot learning to paint.",
    system_prompt="You are a creative writer. Be vivid but concise.",
    max_tokens=150
):
    print(token, end="", flush=True)

print()

## 5. Detailed Responses

Get token usage and metadata along with the response.

In [ ]:
response = client.chat_with_details(
    message="What is Python?",
    max_tokens=100
)

print("=== Response Details ===")
print(f"Content: {response.content}")
print(f"\nModel: {response.model}")
print(f"Finish reason: {response.finish_reason}")
print(f"\nToken usage:")
print(f"  - Input tokens: {response.usage['input_tokens']}")
print(f"  - Output tokens: {response.usage['output_tokens']}")

## 6. Claude's Strengths

Some tasks where Claude particularly excels.

In [ ]:
# Complex instruction following
response = client.chat(
    message="Explain quantum entanglement",
    system_prompt="""Format your response as:
## Simple Explanation
[1-2 sentences a child could understand]

## Technical Details
[2-3 sentences with proper physics terminology]

## Real-world Application
[1 practical use case]"""
)
print(response)

In [ ]:
# Structured output
response = client.chat(
    message="List 3 popular programming languages",
    system_prompt="Respond only in valid JSON format. No markdown code blocks, just raw JSON.",
    temperature=0.0
)
print(response)

In [ ]:
# Thoughtful analysis
response = client.chat(
    message="What are the pros and cons of remote work?",
    system_prompt="Be balanced and consider multiple perspectives. Keep it concise with 2-3 points each.",
    max_tokens=300
)
print(response)

## Summary

You've learned:

1. **Basic usage**: `client.chat(message)` for simple queries
2. **System prompts**: Claude excels at following detailed instructions
3. **Multi-turn**: Maintain context with `history` parameter
4. **Streaming**: Real-time output with `client.stream()`
5. **Details**: Get usage stats with `client.chat_with_details()`

## Next Steps

- Try the [Gemini notebook](03_gemini.ipynb) to compare with Google's model
- Check out [06_comparison.ipynb](06_comparison.ipynb) for side-by-side comparisons
- Explore Claude's long context capabilities with larger documents